In [6]:
pip install --pre deepmimo

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 KB 753.0 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
import DeepMIMOv3
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
import time
import math
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import IterableDataset
import numpy as np
import time, gc
from tqdm import tqdm
import numpy as np
import torch
import random
import torch.nn as nn
# from lwm_model import lwm
from torch.optim import Adam
from pathlib import Path
import torch, time



In [3]:
# GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
import torch
print(torch.version.cuda)                   
print(torch.backends.cudnn.version())       
print("CUDA available:", torch.cuda.is_available())  # True

12.6
90501
CUDA available: True


## dataset denver download if you download than pass

In [35]:
import deepmimo as dm

# Search for scenarios matching criteria
scenarios = dm.search()

In [36]:
scenarios

['asu_campus_3p5',
 'city_0_newyork_3p5',
 'city_1_losangeles_3p5',
 'city_2_chicago_3p5',
 'city_3_houston_3p5',
 'city_4_phoenix_3p5',
 'city_5_philadelphia_3p5',
 'city_6_miami_3p5',
 'city_7_sandiego_3p5',
 'city_8_dallas_3p5',
 'city_9_sanfrancisco_3p5',
 'city_10_austin_3p5',
 'city_11_santaclara_3p5',
 'city_12_fortworth_3p5',
 'city_13_columbus_3p5',
 'city_14_charlotte_3p5',
 'city_15_indianapolis_3p5',
 'city_16_sanfrancisco_3p5',
 'city_17_seattle_3p5',
 'city_18_denver_3p5',
 'city_19_oklahoma_3p5',
 'i1_2p5',
 'i2_28b',
 'city_0_newyork_28',
 'city_1_losangeles_28',
 'city_2_chicago_28',
 'city_3_houston_28',
 'city_4_phoenix_28',
 'city_5_philadelphia_28',
 'city_7_sandiego_28',
 'city_6_miami_28',
 'city_8_dallas_28',
 'city_9_sanfrancisco_28',
 'city_10_austin_28',
 'city_11_santaclara_28',
 'city_12_fortworth_28',
 'city_13_columbus_28',
 'city_14_charlotte_28',
 'city_15_indianapolis_28',
 'city_16_sanfrancisco_28',
 'city_17_seattle_28',
 'city_18_denver_28',
 'city_

In [37]:
import deepmimo as dm

# Download a specific scenario
dm.download('city_18_denver_28')

Scenario "city_18_denver_28" already exists in /home/dlghdbs200/LWM_denver/deepmimo_scenarios


# dataset load

In [38]:
import deepmimo as dm

# Load dataset
dataset = dm.load("city_18_denver_28")

# Access dataset properties
aoa_az = dataset.aoa_az
aoa_el = dataset.aoa_el
inter_pos = dataset.inter_pos

# Compute specific channel information
# los = dataset.compute_los()
channels = dataset.compute_channels()
pl = dataset.compute_pathloss()

Loading TXRX PAIR: TXset 1 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 2 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 3 (tx_idx 0) & RXset 0 (rx_idxs 43248)


Generating channels: 100%|█████████████████████████████████████████████████████| 43248/43248 [00:01<00:00, 26819.05it/s]


In [53]:
channels[0].shape

(43248, 1, 8, 1)

In [57]:
pl[0].shape

(43248,)

In [61]:
channels[0][0][0]

array([[-4.0636469e-10-2.4687419e-10j],
       [ 4.0486328e-10+2.4921543e-10j],
       [-4.0334483e-10-2.5152960e-10j],
       [ 4.0180989e-10+2.5381652e-10j],
       [-4.0025908e-10-2.5607602e-10j],
       [ 3.9869291e-10+2.5830790e-10j],
       [-3.9711204e-10-2.6051203e-10j],
       [ 3.9551701e-10+2.6268823e-10j]], dtype=complex64)

In [67]:
pip list

Package                   Version
------------------------- --------------
ace_tools                 0.0
anyio                     4.9.0
argon2-cffi               25.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
Automat                   20.2.0
babel                     2.17.0
bcrypt                    3.2.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blinker                   1.4
certifi                   2020.6.20
cffi                      1.17.1
chardet                   4.0.0
charset-normalizer        3.4.2
click                     8.0.3
cloud-init                25.1.4
colorama                  0.4.4
comm                      0.2.2
command-not-found         0.3
configobj                 5.0.6
constantly                15.1.0
contourpy                 1.3.2
cpplint                   1.5.5
cryptography              3.4.8
cycler                